<a href="https://colab.research.google.com/github/maneakansha36/my_first_repository/blob/main/etl_akanksha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the dataset into a DataFrame
#  Identify number of records and unique locations
# Print top 5 rows and column names

In [37]:
import pandas as pd
import sqlite3

# Load dataset
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
df = pd.read_csv(url)

# Number of records
print("Total records:", len(df))

# Unique locations (place column)
print("Unique locations:", df['place'].nunique())

# Column names
print("Column names:", df.columns.tolist())

# Top 5 rows
print(df.head())



Total records: 9912
Unique locations: 5249
Column names: ['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst', 'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type', 'horizontalError', 'depthError', 'magError', 'magNst', 'status', 'locationSource', 'magSource']
                       time   latitude   longitude  depth   mag magType   nst  \
0  2025-09-21T10:29:09.580Z  33.535999 -116.718002   6.14  0.96      ml  36.0   
1  2025-09-21T10:20:13.414Z  62.887000 -148.835800  62.50  1.30      ml   NaN   
2  2025-09-21T10:16:53.678Z  38.026000 -118.575800   0.00  1.40      ml  11.0   
3  2025-09-21T10:13:02.706Z  60.093100 -152.891000  95.30  0.90      ml   NaN   
4  2025-09-21T10:09:29.210Z  38.830002 -122.804497   1.94  0.70      md  10.0   

      gap      dmin     rms  ...                   updated  \
0   42.00  0.033460  0.1400  ...  2025-09-21T10:31:31.762Z   
1     NaN       NaN  0.3600  ...  2025-09-21T10:21:50.080Z   
2  175.03  0.232000  0.3322  ...  2025-09-2

# **Convert time to datetime**

In [ ]:
df['time'] = pd.to_datetime(df['time'])


# Drop missing values in key columns

In [ ]:
df = df.dropna(subset=['latitude', 'longitude', 'mag'])


#Filter earthquakes with magnitude ≥ 4.0

In [ ]:
df = df[df['mag'] >= 4.0]


#Add a column for day_of_week

In [ ]:
df['day_of_week'] = df['time'].dt.day_name()


# Create a severity_level column
< 4.0 → Low

4.0 - 6.0 → Moderate

6.0+ → High

In [ ]:
def severity(mag):
    if mag < 4.0:
        return "Low"
    elif 4.0 <= mag < 6.0:
        return "Moderate"
    else:
        return "High"

df['severity_level'] = df['mag'].apply(severity)


# Count earthquakes per place

In [ ]:
eq_per_place = df['place'].value_counts().reset_index()
eq_per_place.columns = ['place', 'earthquake_count']


# Compute average magnitude & max depth per day

In [ ]:
summary = df.groupby(df['time'].dt.date).agg(
    avg_magnitude=('mag', 'mean'),
    max_depth=('depth', 'max')
).reset_index()

summary.rename(columns={'time': 'date'}, inplace=True)


# Save to CSV

In [ ]:
df.to_csv("cleaned_earthquakes.csv", index=False)
summary.to_csv("earthquake_summary.csv", index=False)


# Store into SQLite

In [ ]:
import sqlite3

conn = sqlite3.connect("earthquakes.db")

# Save main cleaned dataset
df.to_sql("earthquakes", conn, if_exists="replace", index=False)

# Save summary table
summary.to_sql("earthquake_summary", conn, if_exists="replace", index=False)

conn.close()


In [46]:
conn.commit()  # Save changes
conn.close()   # Close connection


             ┌─────────────────────┐
             │   Extract Data      │
             │  (USGS CSV online)  │
             └─────────┬───────────┘
                       │
                       ▼
             ┌─────────────────────┐
             │   Transform Data    │
             │ - Convert time      │
             │ - Drop nulls        │
             │ - Filter mag >= 4   │
             │ - Add day_of_week   │
             │ - Add severity      │
             │ - Aggregations      │
             └─────────┬───────────┘
                       │
                       ▼
             ┌─────────────────────┐
             │     Load Data       │
             │ - Save CSV files    │
             │ - Save SQLite DB    │
             └─────────────────────┘
